This notebook compares data I could find online for the index of refraction of CXRO data, data from David Windt, and the data files we currently have on volta for Al, Si, AlF$_3$, and SiO$_2$.

The data are stored with different units and assumptions, so I wanted to detail the conversions I used here first.

# Base Units
The base units I will use are wavelength in nanometers (nm) and the index of refraction in terms of $\delta$ and $\beta$. Other units will be converted to these units as needed.

* 1 Angstrom = 0.1 nm
* 1 nm = 1239.84197 eV-nm/$E = hc/E$ , where $E$ is the energy. With $E$ in eV, $h=6.626176\times 10^{-34}$J-s, $c=2.99792458\times 10^8$m/s, $e=1.6021872\times 10^{-19}$C, $hc=1.9864458\times 10^{-25}$J-m = $1.9864458\times 10^{-16}$J-nm. $hc/e=1239.84197$eV-nm.
* $\delta  =\frac{r_0 N}{2\pi} \lambda^2 f_1 = 2.7008644\times 10^{-4} \hbox{nm}^{-2} \frac{\rho\lambda^2}{A}f_1$ with $N$ the density, and $r_0$ the classical electron radius, $2.817940\times 10^{-6}$ nm. If $\rho$ is the density in g/cm$^3$, $A_0=6.0221409\times 10^{23}$ is Avogadro's number, and $A$ g/mole is the atomic mass number $\delta =  2.7008644\times 10^{-4} \hbox{nm}^{-2} \frac{\rho\lambda^2}{A}f_1$
* $\beta = \frac{r_0 N}{2\pi} \lambda^2 f_2 = 2.7008644\times 10^{-4} \hbox{nm}^{-2} \frac{\rho\lambda^2}{A}f_2$
* $f_1 = \frac{2\pi}{r_0 N} \lambda^{-2} \delta = 3702.518 \hbox{ nm}^2 \frac{A}{\rho\lambda^2 } \delta$
* $f_2 = \frac{2\pi}{r_0 N} \lambda^{-2} \beta = 3702.518 \hbox{ nm}^2 \frac{A}{\rho\lambda^2 } \beta$
* $\tilde{n}=1-\delta+i\beta=n+ik$ (complex index of refraction)

Here are the routines I use for unit conversion

In [1]:
delta(rho, lambda, A, f1) = 2.7008644e-4*rho*lambda^2/A*f1
beta(rho, lambda, A, f2) = 2.7008644e-4*rho*lambda^2/A*f2
f1(rho, lambda, A, delta) = 3702.518*A/(rho*lambda^2)*delta
f2(rho, lambda, A, beta) = 3702.518*A/(rho*lambda^2)*beta
e2lambda(e) = 1239.84197/e
lambda2e(lambda) = 1239.84197/lambda

lambda2e (generic function with 1 method)

In [2]:
alf1=2.31139
alf2=0.175442
alA=26.982
alrho=2.70
E=30.255
lambda=e2lambda(E)
println("delta = ", delta(alrho, lambda, alA, alf1))
println("beta = ", beta(alrho, lambda, alA, alf2))

delta = 0.10490688443488634
beta = 0.007962772885157992


I checked this conversion against the CXRO tables. For the above values of f1 and f2 it gave delta = 0.1049094 and beta = 0.007962753

In [3]:
using Test

In [4]:
@testset "conversion" begin
    alf1=2.31139 # f2 for Al at 30.2555 eV
    alf2=0.175442 # f2 for Al at 30.2555 eV
    alA=26.982 # atomic mass of Al
    alrho=2.70 # density of Al
    E=30.2555 # energy for this test
    lambda=e2lambda(E)
    @test isapprox(lambda, 40.97906, atol = 1e-5)
    d = delta(alrho, lambda, alA, alf1)
    b = beta(alrho, lambda, alA, alf2)
    @test isapprox(d,0.1049094, atol = 1e-5)
    @test isapprox(b, 0.007962753, atol = 1e-6)
    @test isapprox(f1(alrho, lambda, alA, d), alf1, atol = 1e-6)
    @test isapprox(f2(alrho, lambda, alA, b), alf2, atol = 1e-6)
end;

Test Summary: | Pass  Total
conversion    |    5      5


With compounds, use the sum of the density for each of the atomic components. These functions compute $\delta$ and $\beta$ for binary compounds. The compounds have n1 atoms with atomic mass A1 and n2 atoms with atomic mass A2 in each molecular unit. f1 for atom 1 is f11, f1 for atom2 is f12, f2 for atom 1 is f21 f2 for atom 2 is f22. The contribution of atom1 to delta is delta computed with the molecular weight times the number of atoms of type atom1 in the molecule. The contribution of atom2 to delta is delta computed with the molecular weight times the number of atoms of type atom2 in the molecule.

In [5]:
function delta2(rho, lambda, A1, n1, f11, A2, n2, f12)
    amol = n1*A1+n2*A2
    d1 = delta(rho, lambda, amol, f11) * n1
    d2 = delta(rho, lambda, amol, f12) * n2
    d1+d2
end
function beta2(rho, lambda, A1, n1, f21, A2, n2, f22)
    amol = n1*A1+n2*A2
    b1 = beta(rho, lambda, amol, f21) * n1
    b2 = beta(rho, lambda, amol, f22) * n2
    b1+b2
end

beta2 (generic function with 1 method)

In [6]:
rho = 2.88 # density of AlF3
ff1 = 0.517012 # F f1 at 30.2555 eV
ff2 = 4.96088 # F f2 at 30.2555 eV
fA = 18.998 # F atomic mass
E=30.2555
lambda=e2lambda(E)
println("delta = ", delta2(rho, lambda, alA, 1, alf1, fA, 3, ff1))
println("beta = ", beta2(rho, lambda, alA, 1, alf2, fA, 3, ff2))

delta = 0.060079215657434994
beta = 0.23422526564012883


CXRO computed the values delta = 0.06007458 and beta = 0.2342305, which looks consistent.

In [7]:
@testset "molecular conversion" begin
    alf1=2.31139
    alf2=0.175442
    alA=26.982
    ff1=0.517012
    ff2=4.96088
    fA=18.998
    alf3rho=2.88
    E=30.2555
    lambda=e2lambda(E)
    d = delta2(alf3rho, lambda, alA, 1, alf1, fA, 3, ff1)
    b = beta2(alf3rho, lambda, alA, 1, alf2, fA, 3, ff2)
    @test isapprox(d,0.06007458, atol = 5e-6)
    @test isapprox(b,0.2342305, atol = 6e-6)
end;

Test Summary:        | Pass  Total
molecular conversion |    2      2


Read in the index of refraction from the CXRO data files.

In [18]:
using DataFrames
using CSV
using Interp
data_dir = homedir()*"/Documents/alf3-paper/analysis/CXRO/"

"C:\\Users\\rstur_000/Documents/alf3-paper/analysis/CXRO/"

al_df = CSV.read(data_dir*"/al.nff",delim='\t',header=["E","f1","f2","blank"],datarow=2,footerskip=60);
al_df = al_df[:,1:3]

Note that the files are problematic. The header with the field names has three columns, but the data has four columns. The latest rows only have three columns. I could get the later rows, but they have energies higher than I care about.

In [40]:
f_df = CSV.read(data_dir*"/f.nff",delim='\t',header=["E","f1","f2"], datarow=2, rows=9)
f2_df = CSV.read(data_dir*"/f.nff",delim='\t',header=["E","f1","f2","blank"], datarow=10, rows=402)
f_df = vcat(f_df,f2_df[:,1:3])
f3_df = CSV.read(data_dir*"/f.nff",delim='\t',header=["E","f1","f2"], datarow=403)
f_df = vcat(f_df,f3_df);

This trick could be used to get the rest of the data for Al if I wanted, but I don't.

In [61]:
o_df = CSV.read(data_dir*"/o.nff",delim='\t',header=["E","f1","f2","blank"], datarow=2, rows=388)
o2_df = CSV.read(data_dir*"/o.nff",delim='\t',header=["E","f1","f2"], datarow=390)
o_df = vcat(o_df[:,1:3],o2_df);

In [70]:
si_df = CSV.read(data_dir*"/si.nff",delim='\t',header=["E","f1","f2","blank"], datarow=2, rows=712)
si2_df = CSV.read(data_dir*"/si.nff",delim='\t',header=["E","f1","f2"], datarow=713)
si_df = vcat(si_df[:,1:3],si2_df);

# General Optical Constant Reader
If I need to read a lot of optical constants, it could be good to have a general reader. That's what I'm writing here.

In [19]:
function CXROread(element::AbstractString)
    data_dir = homedir()*"/Documents/alf3-paper/analysis/CXRO/"
    lines = open(data_dir*element*".nff") do f
        readlines(f)
    end
    npts = length(lines)-1
    E = Array{Union{Missing, Float64}}(missing,npts)
    f1 = Array{Union{Missing, Float64}}(missing,npts)
    f2 = Array{Union{Missing, Float64}}(missing,npts)
    for i = 1:npts
        parts = split(lines[i+1])
        E[i] = parse(Float64,parts[1])
        f1[i] = parse(Float64,parts[2])
        f2[i] = parse(Float64, parts[3])
    end
    DataFrame(E=E,f1=f1,f2=f2)
end

CXROread (generic function with 1 method)

In [20]:
al_df = CXROread("al");
f_df = CXROread("f");
o_df = CXROread("o");
si_df = CXROread("si");

# Data Interpolators
Construct index interpolators which can return the index of refraction at any wavelength.


In [ ]:
struct CXRO
    ndata :: Interp.PCHIP
    kdata :: Interp.PCHIP
    nmin :: Float64
    nmax :: Float64
    kmin :: Float64
    kmax :: Float64
end
function CXRO(rho::Float64, A::Float64, fdata :: DataFrame)
    
    